# 项目结构
- The_battle_memory
    - Genichiro_Ashina (内含1场演示用数据集和101场训练用的完整数据集，由于文件过大所以只上传演示用数据集)
        - training_data-1.npy (演示用数据集)
        - training_data-2.npy
        - ... (2 ~ 102 都是训练用的完整数据集)
        - training_data-102.npy

- pysekiro
    - \__init__.py

    - keys.py (键盘按键的信息)
    - get_keys.py (捕获键盘的按键)
    - direct_keys.py (控制键盘的按键)
    - actions.py (动作控制)

    - grab_screen.py (屏幕图像抓取)
    - get_vertices.py (顶点位置获取)
    - get_status.py (状态获取)

    - collect_data.py (收集数据)

    - Agent.py (DQN)

# 数据预览

- The_battle_memory
    - Genichiro_Ashina
        - training_data-1.npy (演示用数据集)

In [1]:
import os

import cv2
import numpy as np

boss = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', boss, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)
print(data.shape)    # 表示当前演示的数据集由200个数据组成，其中每个数据又由图像与按键记录一一对应的数据组成

(200, 2)


## 单个数据的展示

In [2]:
screen = data[0][0]    # 选取了第一个数据的图像部分
print('图像', screen, screen.shape, screen.dtype)

图像 [[ 93  91  91 ...  98  98 112]
 [ 62  53  56 ...  76  83  81]
 [ 72  57  55 ...  86  79  79]
 ...
 [139 160 169 ... 106 116 119]
 [118 153 159 ... 101 115 123]
 [118 144 149 ...  97  92 130]] (270, 480) uint8


In [3]:
action_value = data[0][1]    # 选取了第一个数据的按键记录部分
print('按键记录', action_value)

按键记录 [0, 0, 0, 0, 1]


## 以视频的形式展示数据

In [4]:
Remaining = len(data)
for screen, action_value in data:
    if   action_value == [1,0,0,0,0]:
        action = '攻击'
    elif action_value == [0,1,0,0,0]:
        action = '弹反'
    elif action_value == [0,0,1,0,0]:
        action = '垫步'
    elif action_value == [0,0,0,1,0]:
        action = '跳跃'
    elif action_value == [0,0,0,0,1]:
        action = '其他'

    cv2.imshow('screen', screen)
    cv2.waitKey(5)
    
    Remaining -= 1
    print(f'\r 剩余: {Remaining:>4}, 动作:{action:<11}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

 剩余:    0, 动作:其他         

# pysekiro 模块解析

## 按键部分
- keys.py (键盘按键的信息)
- get_keys.py (捕获键盘的按键)
- direct_keys.py (控制键盘的按键)
- actions.py (动作控制)

### keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/keys.py)

这个文件不是用来直接使用，而是给我们查找相应的键对应的值的。  
其中，检测按键的是 virtual keys，输出按键的是 direct keys，相应代码在第 146 ~ 254 和 35 ~ 143 行。
使用如下

### get_keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_keys.py)

检测按键的部分，用的是 virtual keys 。

其中的字典 vk 是从 keys.py 中截取的部分键值对，然后，调用 key_check() 时就会检测当前正在按的键

运行以下代码体验一下，按 P 停止

In [ ]:
from pysekiro.get_keys import key_check
while True:
    keys = key_check()
    print(f'\r{str(keys):<60}', end = '')
    if 'P' in keys:
        break

### direct_keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/direct_keys.py)

某个大佬写的代码，反正从应用的角度看就好了。

PressKey 压键  
ReleaseKey 松键  

具体用法看 actions.py

### actions.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/actions.py)

输出按键的部分，用的是 direct keys 

和 get_keys.py 不同的是，这里不用字典处理，而是采用直接赋值的方式。  
因为从 direct_keys.py 导入的 PressKey() 和 ReleaseKey() 参数都是 hexKeyCode ，直接输入相应的 十六进制 映射到按键就好了。  
赋值给相应的变量只是为了好认而已。

In [ ]:
# e.g.
from pysekiro.direct_keys import PressKey, ReleaseKey
import time

W = 0x11
def Move_Forward():
    print('移动 前')
    PressKey(W)
    time.sleep(1)
    ReleaseKey(W)

for _ in range(5):
    Move_Forward()

调用其中的某个函数时，游戏中的人物就会做出相应的动作

act() 等之后用的时候再讲

## 图像部分
- grab_screen.py (屏幕图像抓取)
- get_vertices.py (顶点位置获取)
- get_status.py (状态获取)

### grab_screen.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/grab_screen.py)

这也是某个大佬写的，我把输出的图像改成了灰度图 cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY) ，然后又补充了适应这个游戏的预处理而已。  
我指定的区域，正是在显示器左上角一个 1280 x 720 的位置，其中还包括上半部分 30 像素的边框。  
运行一下下面的程序就知道了。

In [5]:
import cv2
from pysekiro.grab_screen import get_screen

screen = get_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

print('图像', screen, screen.shape, screen.dtype)

图像 [[ 24  24  57 ...  23  23  23]
 [ 59  59  59 ...  59  59  59]
 [ 59  59  59 ...  43  43  43]
 ...
 [238 238 238 ... 255 255 255]
 [238 238 238 ... 255 255 255]
 [238 238 238 ... 255 255 255]] (270, 480) uint8


实际上，数据集中单个数据里的图像部分就是由这个函数获取的。

### get_vertices.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_vertices.py)

其实就是我无法写出一个自动把血条、架势条抠出来的程序，所以才写的这个半自动的程序。  
这个程序的主要作用就是，在图像上点击然后自动返回相关的位置信息。  
可以看一下演示的gif  
![demo.gif](https://github.com/ricagj/pysekiro_with_RL/blob/main/imgs/demo.gif)

如图所示，在弹出来的窗口中用鼠标左键按顺序依次点击左下，左上，右上，右下，然后按键盘上的“ESC”键，就会自动返回 x, x_w, y, y_h, vertices  
**图像的原点在左上角**
- x 代表 x轴的起始位置，也是图像的最左边
- y 代表 y轴的起始位置，也是图像的最上边
- x_w 代表 x轴的终止位置，也是图像的最右边
- y_h 代表 y轴的终止位置，也是图像的最下边
- vertices = [[x, y_h], [x, y], [x_w, y], [x_w, y_h]] (依次代表图像的左下角、左上角、右上角和右下角的位置。)

#### 函数解释

##### on_EVENT_LBUTTONDOWN(event, x, y, flags, param)

vertices 正是由下面这些代码获取的
~~~python
cv2.setMouseCallback("window", on_EVENT_LBUTTONDOWN)
while True:
    cv2.imshow("window", img)
    if cv2.waitKey(0)&0xFF==27:
        break
~~~
具体用法看下面

##### GrabCut_ROI(img, vertices)

其实就是展示抠图的效果的，看上面的 demo 就知道了，区域内保留，区域外全黑（像素点的值等于0）。

##### standardize(vertices)

鼠标点击出来的位置只是大概的，再怎么精细也只是类似矩形，而不是矩形，所以靠这个函数处理成最标准的矩形。  
标准：尽量取最大范围

##### roi(img, x, x_w, y, y_h)

只保留区域内，区域外的全部不要

##### get_vertices(img)

直接上代码解释

~~~python
def get_vertices(img):
    
    global vertices
    vertices = []

    print('Press "ESC" to quit. ') # 按ESC键离开。
    cv2.namedWindow("window", cv2.WINDOW_NORMAL)    # 使窗口可以调整，比如图片太小，就调整窗口
    cv2.setMouseCallback("window", on_EVENT_LBUTTONDOWN)
    while True:
        cv2.imshow("window", img)
        if cv2.waitKey(0)&0xFF==27:
            break
    cv2.destroyAllWindows()

    if len(vertices) != 4:    # 矩形嘛，4个点不是正常的吗
        print("vertices number not match")
        return -1

    x, x_w, y, y_h, vertices = standardize(vertices)    # 矩形化

    cv2.imshow('img', img)
    cv2.imshow('GrabCut_ROI(img)', GrabCut_ROI(img, [np.array(vertices)]))    # GrabCut_ROI 效果
    cv2.imshow('roi(img)', roi(img, x, x_w, y, y_h))    # roi 效果

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f'\n x={x}, x_w={x_w}, y={y}, y_h={y_h}, vertices={vertices} \n')
~~~

#### 实战

In [6]:
import cv2
from pysekiro.get_vertices import get_vertices

img = cv2.imread(".\imgs\demo.png", 0)
get_vertices(img)

Press "ESC" to quit. 

 x=68, x_w=499, y=677, y_h=696, vertices=[[68, 696], [68, 677], [499, 677], [499, 696]] 



其实最后需要用的只有 x, x_w, y, y_h

### get_status.py 注：仍需优化

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_status.py)

#### 实战

In [7]:
import os
import cv2
import numpy as np
from pysekiro.get_status import get_status

boss = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', boss, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)

Remaining = len(data)

for screen in data[:,0]:

    cv2.imshow('screen', screen)

    Sekiro_HP, Sekiro_Posture, Boss_HP, Boss_Posture = get_status(screen)

    Remaining -= 1
    print(f'\r Remaining: {Remaining:<6}, Sekiro_HP: {Sekiro_HP:>4}, Sekiro_Posture: {Sekiro_Posture:>4}, Boss_HP:{Boss_HP:>4}, Boss_Posture: {Boss_Posture:>4}', end='')
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):    # 按 q 键关闭视频
        cv2.destroyAllWindows()
        break
else:
    cv2.destroyAllWindows()

 Remaining: 0     , Sekiro_HP:  117, Sekiro_Posture:   24, Boss_HP:  64, Boss_Posture:   58

#### 解释

其实和最上面那个 以视频的形式展示数据 类似，只不过内容换成了检测状态

关键部分
~~~python
Sekiro_HP, Sekiro_Posture, Boss_HP, Boss_Posture = get_status(screen)
~~~

get_status 函数原型
~~~python
def get_status(img):
    return [get_Sekiro_HP(img), get_Sekiro_Posture(img), get_Boss_HP(img), get_Boss_Posture(img)]
~~~

里面四个函数的原型
~~~python
def get_Sekiro_HP(img):
    img_roi = roi(img, x=29, x_w=182, y=244, y_h=246)[0]    # 我方血条的区域，通过 get_vertices.py 中的 get_vertices(img) 获取
    Sekiro_HP = get_HP(img_roi)    # 转换为数值，通过 get_HP(target_img)
    return Sekiro_HP
~~~
~~~python
def get_Sekiro_Posture(img):
    img_roi = roi(img, x=241, x_w=290, y=233, y_h=235)[0]    # 我方架势条的区域，通过 get_vertices.py 中的 get_vertices(img) 获取
    Sekiro_Posture = get_Posture(img_roi)    # 转换为数值，通过 get_Posture(target_img)
    return Sekiro_Posture
~~~
~~~python
def get_Boss_HP(img):
    img_roi = roi(img, x=29, x_w=129, y=24, y_h=26)[0]    # 敌方血条的区域，通过 get_vertices.py 中的 get_vertices(img) 获取
    Boss_HP = get_HP(img_roi)    # 转换为数值，通过 get_HP(target_img)
    return Boss_HP
~~~
~~~python
def get_Boss_Posture(img):
    img_roi = roi(img, x=241, x_w=326, y=16, y_h=18)[0]    # 敌方架势条的区域，通过 get_vertices.py 中的 get_vertices(img) 获取
    Boss_Posture = get_Posture(img_roi)    # 转换为数值，通过 get_Posture(target_img)
    return Boss_Posture
~~~

可以看出四个函数的结构是相同的，只不过是计算的区域不同而已，  
然后，img_roi 只取第一行的数据，是因为每行的数据都是差不多的，误差不会很大，所以只需要取其中的一行的结果就可以了，  
当然，只是目前采取这种做法而已，以后说不定为了提高抗干扰性而使用更多行数据。

获取数值的部分
~~~python
def get_HP(target_img):
    count = 0
    
    if target_img[0] == 0 or target_img[1] == 0:
        return count
    
    for i in range(len(target_img)-1):
        cur_pixel = int(target_img[i])
        next_pixel = int(target_img[i+1])
        if abs(cur_pixel - next_pixel) > 20 or cur_pixel < 40 or cur_pixel > 80:
            break
        count += 1
    return count
~~~
~~~python
def get_Posture(target_img):
    count = 0
    
    if target_img[0] == 0 or target_img[1] == 0:
        return count
    
    for i in range(len(target_img)-1):
        cur_pixel = int(target_img[i])
        next_pixel = int(target_img[i+1])
        if abs(cur_pixel - next_pixel) > 20 or cur_pixel < 100:
            break
        count += 1
    return count
~~~
这两个也是基本相同的，不同的是，停止的条件  
- abs(cur_pixel - next_pixel) > 20 or cur_pixel < 40 or cur_pixel > 80
- abs(cur_pixel - next_pixel) > 20 or cur_pixel < 100
*这两个只是目前这样设置的，以后会更改*  

运行下面的代码

In [ ]:
import cv2
from pysekiro.get_vertices import roi
img = cv2.imread(".\imgs\demo.png", 0)
cv2.imshow('img', roi(img, x=70, x_w=355, y=85, y_h=106))
cv2.waitKey(0)

上面的代码给我们展示的是截取的生命栏的图像，或者看下面的图片也可以  
![demo.png](https://github.com/ricagj/pysekiro_with_RL/blob/main/imgs/demo.png)  
仔细观察，可以发现它由五部分组成：
- 生命栏开始的位置是一条白线
- 红色的现有生命值（灰度图看不出来）
- 现有的生命与已失去的生命之间是一条白线
- 灰色的已失去的生命值
- 生命栏结束的位置是一条白线

接下来看下面的代码运行的结果

In [8]:
import os
import numpy as np
from pysekiro.get_vertices import roi
from pysekiro.get_status import get_HP

def get_Boss_HP(img):
    img_roi = roi(img, x=29, x_w=129, y=24, y_h=26)[0]
    Boss_HP = get_HP(img_roi)
    print(img_roi)
    return Boss_HP

boss = 'Genichiro_Ashina' # 苇名弦一郎
path = os.path.join('The_battle_memory', boss, f'training_data-{1}.npy')
data = np.load(path, allow_pickle=True)

for screen in data[88:93,0]:

    Boss_HP = get_Boss_HP(screen)

    print(f'Boss_HP: {Boss_HP:>4}', end='\n\n\n')

[57 59 52 56 56 54 58 50 51 57 52 56 58 54 48 55 54 55 50 58 55 53 53 60
 49 53 58 59 51 56 50 60 52 56 60 49 58 53 55 54 49 52 51 49 49 55 49 57
 56 58 49 49 56 52 56 47 56 49 53 57 53 54 53 52 47 57 53 54 55 88 51 44
 43 40 43 45 30 46 46 37 39 40 38 39 41 37 41 44 38 34 35 42 44 48 41 35
 30 33 41 37]
Boss_HP:   68


[57 59 52 56 56 54 58 50 51 57 52 56 58 54 48 55 54 55 50 58 55 53 53 60
 49 53 58 59 51 56 50 60 52 56 60 49 58 53 55 54 49 52 51 49 48 55 49 57
 56 58 49 49 56 52 56 47 56 49 53 57 53 54 53 52 47 57 53 54 55 88 51 44
 43 40 43 45 30 46 46 37 39 40 38 39 41 37 41 44 38 34 35 42 44 48 41 34
 30 34 41 37]
Boss_HP:   68


[ 57  59  52  56  56  54  58  50  51  57  52  56  58  54  48  55  54  55
  50  58  55  53  53  60  49  53  58  59  51  56  50  60  52  56  60  49
  58  53  55  54  49  52  51  49  48  55  49  57  56  58  49  49  56  52
  56  47  56  49  53  57  53  54  53  52  47  57  53  74 114 101  38  44
  43  40  43  45  30  46  46  37  39  40  38  38  41  37  41  44

**注：我目前截取的生命栏不包括开始和结束**  
不包括开始和结束的话，截取的生命栏只由三部分组成，现有生命值、间隔和已失去的生命值。  
左边的颜色比较浅，中间的间隔部分颜色最浅，然后右边的颜色偏深。  
所以左边的像素点数值的范围大概在 40 ~ 60 之间，遇到中间的间隔后，间隔处的像素点数值迅速增大到80以上，然后再迅速减小到40以下。  
迅速变化的范围大于20，所以当**下一个像素点的数值**和**当前像素点的数值**的差值大于 20 ，或者像素值大于 80 (间隔的位置)，又或者像素值小于 40 (已失去的生命值的位置)时停止。
~~~python
if abs(cur_pixel - next_pixel) > 20 or cur_pixel < 40 or cur_pixel > 80:
    break
~~~
然后算法的原理很简单，就是用count计数，然后遍历这个numpy数组，遇到符合条件的 count+=1 ，直到遇到结束条件，就这样，得到的count的值就是最后我们需要的数值。

## 收集数据部分（深度学习必备，强化学习可选）

### collect_data.py

整个代码的核心也就下面这些
~~~python
screen = get_screen()    # 屏幕图像获取
keys = key_check()       # 按键检测
output, action = keys_to_output(keys)     # 按键转化为对应的输出
training_data.append([screen, output])    # 把图像和按键打包在一个列表内，再添加进 training_data
np.save(save_path, training_data)    # 收集到一定程度就保存
~~~

#### 主要代码

~~~python
def collect_data(boss):
    
    # 保存数据的容器
    training_data = []
    
    # 建立临时保存数据的文件夹
    path_1 = 'tmp_data'
    if path_1 not in os.listdir():
        os.mkdir(path_1)
    
    # 初始化保存路径
    starting_value = 1
    save_path = os.path.join(path_1, f'training_data-{starting_value}.npy')
    
    print('Ready!')

    paused = True    # 初始状态是暂停，等准备好了按 T 就直接开始了
    while True:
        
        if not paused:
            last_time = time.time()
            
            screen = get_screen()    # 屏幕图像获取
            keys = key_check()    # 按键检测
            output, action = keys_to_output(keys)     # 按键转化为对应的输出
            
            # 数据整合（打包图像和按键输出）
            training_data.append([screen, output])
            
            # 临时保存数据
            if len(training_data) == 100:    # 每收集100个数据就保存到临时目录
                np.save(save_path, training_data)
                training_data = []    # 重置保存的容器
                starting_value += 1
                save_path = os.path.join(path_1, f'training_data-{starting_value}.npy')    # 更新保存的路径
            
            # 展示这一轮所耗时间，当前按下的键所代表的动作，当前被检测到的按键的列表
            battle_log = f'\rloop took {round(time.time()-last_time, 3):>5} seconds. action {action:<11}. keys {str(keys):<60}'
            print(battle_log, end='')
        
        # 再次检测按键
        keys = key_check()
        if 'P' in keys:      # 按 ‘P’ 结束并保存
            np.save(save_path, training_data)
            break
        elif 'T' in keys:    # 按 ‘T’ 暂停或继续
            if paused:
                paused = False
                print('\nStarting!')
                time.sleep(1)
            else:
                paused = True
                print('\nPausing!')
                time.sleep(1)
    
    print('\n\nStop, please wait')
    
    merge_data(boss)    # 合并临时数据

    print('Done!')
~~~

#### 次要代码 1

In [ ]:
# 代码比较简单，运行看一下结果就好了，注意，只有 get_keys.py 内定义了的才能检测得到
# 目前可以检测的是 ['W', 'S', 'A', 'D', 'J', 'K', 'SPACE', 'LSHIFT', 'T', 'P']
# 但是下面的程序只定义了 'J', 'K', 'LSHIFT', 'SPACE', 所以只会显示那些
from pysekiro.get_keys import key_check

# 手动独热编码，以后方便控制
j  = [1,0,0,0,0] # 攻击 | Attack
k  = [0,1,0,0,0] # 弹反 | Deflect
ls = [0,0,1,0,0] # 垫步 | Step Dodge
sp = [0,0,0,1,0] # 跳跃 | Jump
ot = [0,0,0,0,1] # 其他 | Other

def keys_to_output(keys):

    if   'J' in keys:
        output = j
        action = 'Attack'     # 攻击
    elif 'K' in keys:
        output = k
        action = 'Deflect'    # 弹反
    elif 'LSHIFT' in keys:
        output = ls
        action = 'Step Dodge' # 垫步
    elif 'SPACE' in keys:
        output = sp
        action = 'Jump'       # 跳跃
    else:
        output = ot
        action = 'O'      # 其他
    return output, action

while True:
    keys = key_check()
    output, action = keys_to_output(keys)
    print(f'\r 输出：{str(output)}，动作：{action:<10}', end = '')
    
    if 'P' in keys:    # 停止条件：按下 P 键
        print('结束')
        break

#### 次要代码 2 （这部分完全可以不看，就是保存数据集而已）

find_max_num() 和 merge_data() 是保存数据时用的代码  
临时数据都保存在 tmp_data 下，合并之后的数据保存在 The_battle_memory 相应的boss名称的文件夹内，文件名都是 training_data-**数字**.npy 的格式
用 find_max_num() 找到 tmp_data 内 .npy 文件的最大**数量**，作为遍历结束的条件（tmp_data 内的文件是按顺序的，所以最大数字也代表最大数量）  
用 find_max_num() 找到 The_battle_memory 相应的boss名称的文件夹内 .npy 文件的最大**数字**，取其下一位，能避免文件名重复的情况（数据命名原本是连续的，当偶尔会删掉中间几个质量很差的数据集，就造成了保存的时候文件名不连续的情况，这时候最大数值不能代表最大数量）  
- e.g.
    - training_data-3.npy, training_data-13.npy, training_data-101.npy
    - find_max_num()结果是 101

指定一个路径，找到 .npy 后缀的文件中最大的数字  
比如 tmp_data 文件夹下有 training_data-1.npy ~ training_data-22.npy ，那么输出就是 22
~~~python
def find_max_num(path):
    filenames = os.listdir(path)
    if 'training_data-1.npy' in filenames:
        max_num = max([int(x[14:-4]) for x in filenames if '.npy' in x])
    else:
        max_num = 0
    return max_num
~~~

~~~python
def merge_data(boss):
    path_1 = 'tmp_data'
    max_num_1 = find_max_num(path_1)    # 找到 tmp_data 文件夹下有 .npy 后缀的文件夹最大的数字
    if max_num_1 <= 1:    # 如果小于等于1 那说明数据太少，不被允许保存，删除文件后强制退出
        print('There is no data to merge.')
        shutil.rmtree(path_1)
        return -1
    
    npy_file = os.path.join(path_1, 'training_data-1.npy')
    data = np.load(npy_file, allow_pickle=True)    # 读取第一个数据集
    
    for i in range(2, max_num_1 + 1):    # 准备遍历剩下的数据集
        npy_file = os.path.join(path_1, f'training_data-{i}.npy')
        next_data = np.load(npy_file, allow_pickle=True)    # 读取下一个数据集
        
        data = np.append(data, next_data, axis=0)    # 把这个数据添加到 data 内，实现合并数据集
    
    path_2 = os.path.join('The_battle_memory', boss)
    max_num_2 = find_max_num(path_2)
    np.save(os.path.join(path_2, f'training_data-{max_num_2+1}.npy'), data)    # 保存到相应的位置
    
    shutil.rmtree(path_1)    # 数据已经合并完毕并转移，删除临时文件夹 tmp_data 以及里面的内容
~~~

#### 实战

当提示 Ready! 时，第一次按 T 开始搜集数据，之后按 T 切换暂停和继续的状态，按 P 结束

In [ ]:
from pysekiro.collect_data import collect_data
boss = 'Genichiro_Ashina' # 苇名弦一郎
collect_data(boss)

# 待更新